# Analysis of Su1Vx7
```
Analysis / figures of results from h_DsRnn_Prep_spa.ipynb
Summarizing mult. RNNs / data sets based on Pandas ExTb and ProcTb.

PREC: h_DsRnnAna_Pd.iphnb
```

```
jupyter nbconvert --TemplateExporter.exclude_raw=True --TemplateExporter.exclude_input=True --to html --output tmp h_DsRnnAnaPd_Su1Vx7.ipynb

jupyter nbconvert h_DsRnnAnaPd_Su1Vx7.ipynb \
--to html \
--output tmp \
--TemplateExporter.exclude_raw=True \
--TemplateExporter.exclude_input=True \
--TagRemovePreprocessor.remove_cell_tags="{'rmCell'}" \
--TagRemovePreprocessor.remove_all_outputs_tags="{'rmOut'}" \
--RegexRemovePreprocessor.patterns="[r'(?ms).*DeprecationWarning.*']" \
--RegexRemovePreprocessor.patterns="[r'(?ms)Using TensorFlow.*']" \
```

Add this cell metadata (JSON) e.g: `{ "tags": ["rmOut"]}`

# Imports

In [ ]:
assert False, 'Deprication Warning! Superceded by h_DsRnnAnaPd_Pgrp.ipynb'

In [ ]:
%run h_DsRnn_def.ipynb

In [ ]:
import pandas as pd
%matplotlib inline
# rcPar = mpl.rcParamsDefault
rcPar = dict(mpl.rcParams) # special for %matplotlib inline!
hplotstyles = lambda : [ plt.style.use('seaborn-talk'), mpl.rcParams.update({'figure.figsize': (18,4)}) ];
# %matplotlib notebook
plt.style.use('default') # necessary reset?
hplotstyles();
# plt.style.use('seaborn-talk') # 'default', 'dark_background', 'seaborn-talk'


# ProcTb: Load model parameter sets

In [ ]:
%run h_DsRnn_ProcTb.ipynb
# ProcTb = ProcTb.filter(like='Su1Vx7')
ProcTb.shape


In [ ]:
# Disp example pd.Series
ProcTb.iloc[:,1]


## [ Load ExTb - need this? ]


## Load models / ProcDirs

In [ ]:
os.chdir(ProcTb.iloc[:,1].DataDir)
# os.chdir(ProcTb.iloc[:,1].DataDir+'Proc06/') # FIXIT: Not using ProcDir
# os.chdir(ProcTb.iloc[:,1].DataDir+'Proc05/') # FIXIT: Not using ProcDir
os.getcwd()

In [ ]:
Fname = hsys('find Proc06/Proc*/Km_Su1Vx7_* -iname "EvalData.mat"') # ***
Fname = hsys('find Proc*/Proc_Km_Su0Vs1 -iname "EvalData.mat"') # ***
# Fname = hsys('find Proc06/Proc_Km_Su?V?? -iname "EvalData.mat"') # *** comp Su0Vs1 Su1Vx7
#Fname = hsys('find Proc06/Proc*/Km_Su1Vx7_* Proc*/Proc_Km_Su0Vs1 -iname "EvalData.mat"') # ***
# Fname = hsys('find Proc*/*/Km_Su0Vs1Rs* -iname "EvalData.mat"') # ***
#< Fname = hsys('find Proc*/*/Km_Su0Vf20Xn* -iname "EvalData.mat"') # ***
# Fname = hsys('find Proc_* Km* -iname "model.h5"')
# Fname = hsys('find Proc_* Km* -iname "EvalData.mat"')
#Fname = hsys('find -L Proc_TVdata -iname "EvalData.mat"')
# Fname = hsys('find Proc*/*/Km_Su1Vx7Rper_* -iname "EvalData.mat"')
Fname = sorted( Fname, key=os.path.getctime, reverse=False) #[0]
Fname = list(map(hFpath,Fname))
len(Fname)
Fname


In [ ]:
### Filter Fname by ProcId
ProcId = ('','Vx1','Vs1','Su1Vxn','D1Su1Vxn','Su0Vs1','Su1Vx7_','Su1Vx7Drop_','Vs1Xn','Vx1Xn')[0]
ProcId = ('','Su1Vx7_','Su1Vf20Xn0_','Su0Vs1_')[0]
# ProcId = ('Su1Vr3_','Su1Vr3lin_','Su1Vr3')[0]
# Fname = hsys('find Km_%s_*/ -iname "EvalData.mat"'%ProcId ) # Vs1
if ProcId: Fname = [ n for n in Fname if ProcId in n]
# if ProcId: Fname = filter(lambda x: ProcId in x, Fname)

In [ ]:
Fname = list(map(hFpath,Fname))
Fname = sorted(Fname)
assert len(Fname), 'Oops! No files?!'
print( '\n'.join( Fname[:8])+'\n... total %u'%len(Fname))

In [ ]:
%%time
Pars = hLoadProcDir( Fname)

## Convert Pars to Pandas (ProcTb or SecTb)

In [ ]:
Pars[0].keys()

## Check training success

In [ ]:
for par in Pars:
    tmp = par['TrainLog'].val_loss.values
    n = tmp.argmin()
    par['TrainMetric'] = ( tmp[n] - tmp[0] ) / np.diff(tmp[:n]).std()
    par['BestValEpoch'] = n
    # print('%u \t%f'%(n,par['TrainMetric']))


In [ ]:
#np.array( [ par['TrainMetric'] for par in Pars])
plt.stem(np.array( [ par['TrainMetric'] for par in Pars]), use_line_collection=True);
plt.ylabel('TrainMetric [$\Delta$ Loss / SD ]');
plt.xlabel('models');


In [ ]:
hylimmax = lambda x=(0.5,1.5): plt.ylim( max(plt.ylim()[0],x[0]),min(plt.ylim()[1],x[1]) )

fig,ax = plt.subplots(1,1, figsize=(6,6))
for n in range(len(Pars)):
    x = Pars[n]['TrainLog']['val_loss']
    plt.plot( x/x[0] )

# plt.ylim(0.5,1.5)
hylimmax((0.5,1.5))
if len(Pars)<9: plt.legend(range(len(Pars)));

-------------------------------------
### Compare training success across model parameters (ProcGroups)
#### Notes:
 * Su1Vr3 and Su1Vr3lin show little difference in average TrainMetric
   * ...except for 3 neg. outliers (better) in the former.

In [ ]:
# [ tuple( par[k] for k in ('ProcId','TrainMetric') ) for par in Pars ]
# [ { k:par[k] for k in ('ProcId','TrainMetric') } for par in Pars ]
# Tb = pd.DataFrame( Pars )[['ProcId','TrainMetric']]
Tb = pd.DataFrame( Pars, columns=['ProcId','TrainMetric','BestValEpoch']) #.set_index('ProcId',drop=False)
# Tb['ProcGroup'] = Tb.ProcId.str.split('_',1,expand=True)
Tb[['ProcGroup','ProcId']] = Tb.ProcId.str.split('_',1,expand=True)
Tb = Tb.set_index(['ProcGroup','ProcId'])
# Tb.head()


In [ ]:
# HOWTO Pandas boxplot - by=boxes, column=axes
Tb.boxplot(column='TrainMetric', by='ProcGroup', figsize=(3,4), rot=20);
plt.suptitle(None); plt.title(None); plt.ylabel('TrainMetric')
# plt.ylim(-10,0)
# Tb.get_loc( Tb.TrainMetric < 10 )
Tb[ Tb.TrainMetric < -10 ] 


## Make Xarray from Pars (~SecTb)
[Scratch](#Scratch)

In [ ]:
## Make a MultiIndex for Su*Sec
import xarray as xr

idx = [ ( par['SuId'], n) for par in Pars for n in range(par['x'].shape[0]) ]
idx = pd.MultiIndex.from_tuples(idx,names=['Su','Sc'])

Data = np.concatenate( [ np.concatenate([par['x'],par['y'],par['yh']],-1) for par in Pars ], 0 )
Data = xr.DataArray( Data, dims=['Sec','t','Ch'], \
                    coords={'Ch':['card','resp','mask','Gsig','y','yh'], 'Sec':idx})

Data.shape
# Data

In [ ]:
ValTag = [ par['ValTag'] for par in Pars ] # TODO: store in Par?
assert 1 == len( np.unique(ValTag)), 'Oops! Multiple ValTag numbers!?'

In [ ]:
idx = pd.DataFrame()
for par in Pars:
    tmp = pd.DataFrame( par['SecNp'], columns=['Su','Se','Ex','Sc','Vt'] )
    # tmp['ValTag'] = par['ValTag']
    # tmp = tmp[ tmp.Vt==par['ValTag'] ]
    if par['ValFrac']:
        tmp.Vt = tmp.Vt.apply( lambda x: np.where( x<1, x>=(1-par['ValFrac']), x ))
        tmp.Vt = tmp.Vt.astype(int)

    tmp = tmp[ np.any( np.c_[tmp.Vt.values] == np.r_[par['ValTag']],-1) ]
    tmp = tmp.drop(columns='Vt')
    tmp['ProcGrp'] = par['ProcId'].split('_')[0]
    #< tmp['ProcGrp'] += par['ProcId'][-1]
    idx = idx.append(tmp)

idx = idx.astype(int,errors='ignore')
idx = pd.MultiIndex.from_frame( idx )
# idx = idx.remove_unused_levels() # useless!
if idx.levshape[-1]<2 : idx = idx.droplevel(-1)



In [ ]:
# Data = xr.DataArray( Data, dims=['Sec','t','Ch'], coords={'Ch':['card','resp','mask','Gsig','y','yh'], 'Sec':idx})
Data.coords['Sec'] = idx; # This should work
Data.coords['t'] = np.arange(Data.sizes['t'])/20; # Why does this not quite work properly?

In [ ]:
abs(Data).max(axis=(0,1)) # global max for each channel
tmp = abs(Data).max('t') >= abs(Data).max(['Sec','t'])*0.95 # which sections reach max...
tmp = np.abs(Data).max('t') > 4.01 # which sections reach max...
tmp = tmp[...,:2].any(axis=-1) # ...in any chan.

# HOWTO use Xarray assign_coords
Data = Data.assign_coords(Clipped=tmp) # mark maxes chans in coord. Clipped


In [ ]:
Data.coords

In [ ]:
# HOWTO use multi-dim boolean index on xarray - apply to np.array!
# Data.values[Data[:,:,-2]==0,-2] = np.nan # +++
Data.values[:,np.all( Data[:,:,-2]==0, 0),-2] = np.nan # +++
# Data.values[ np.roll( Data.loc[:,:,'mask'].values==0, -1, -1), -2] = np.nan # +++


In [ ]:
# HOWTO set Matplotlib default font size:

# mpl.rcParams.update({'font.size': 12})
# plt.rc('axes', titlesize=SMALL_SIZE, labelsize=MEDIUM_SIZE)
# mpl.rcParams.update({'axes.labelsize': 16})
# mpl.rcParams.update({'axes.titlesize': 16})
# plt.rcParams.update({'axes.labelsize': 'large'})
# plt.rcParams.update({'axes.titlesize': 'large'})

*****************************************

************************************************************
# Compute Metrics

In [ ]:
# NOTE: Must mask zeros for plotting AND compute par['WRVF']

# NOTE: RVF favors sections that have higher var in y.
hnanrvf = lambda y,yh,dim: np.nanvar( yh-y, dim) / np.nanvar( y, dim)
# hnanrvf = lambda y,yh,dim: np.nanmean( np.abs(yh-y)**2, dim) / np.nanvar( y, dim)


In [ ]:
hnormv = lambda X,d=0: X / np.sqrt(np.nansum(np.abs(X)**2,d,keepdims=True)) # +++
# n = np.any( ~np.isnan(Data.loc[:,:,'y'].values), 0)
# hscalez(Data.loc[:,:,'y'].values[:,n],1) * hscalez(Data.loc[:,:,'yh'].values[:,n],1)
n = np.all( ~np.isnan(Data.values[:,:,-2]), 0)
tmp = np.prod( hscalez( Data.values[:,n,-2:], -2), -1).sum(-1)
#< hpcorr = lambda X: np.prod( hscalez( X[..., np.all(~np.any(np.isnan(X),-1), 0:-2), :], -2), -1).sum(-2)
hpcorr = lambda X: np.nansum( np.prod( hnormv( np.where(np.any(np.isnan(X),-1,keepdims=True),np.nan,X), -2), -1), -1) # +++
#< hrcoef = lambda X: np.nansum( X[:,:,-2] * hnormv( np.where(np.any(np.isnan(X),-1),np.nan,X[:,:,-1]), -2), -1) # +++
hrcoef = lambda X,Y: np.nansum( Y * hnormv( np.where(np.isnan(Y),np.nan,X), -1), -1) # +++
# tmp = hpcorr(Data.values[:,:,-2:])
# plt.plot(tmp)


In [ ]:
def hnancorr(X,Y,d,ddof=None):
    X = np.where(np.isnan(Y),np.nan,X)
    Y = np.where(np.isnan(X),np.nan,Y)
    X -= np.nanmean(X,d,keepdims=True)
    Y -= np.nanmean(Y,d,keepdims=True)
    X /= np.sqrt(np.nansum(X**2,d,keepdims=True))
    Y /= np.sqrt(np.nansum(Y**2,d,keepdims=True))
    if ddof:
        return np.nanmean(X*Y,d)*(X.shape[d]-ddof)
    return np.nansum(X*Y,d)

hpcorr = lambda X: hnancorr( X[...,-1], X[...,-2], -1)


In [ ]:
# HOWTO assign a new attribute var to an Xarray.DataArray
# Data.assign_attrs(wrvf=tmp)

# HOWTO convert Xarray.DataArray to Dataset
# Data = Data.to_dataset(name='Sig')

tmp = hnanrvf( Data[:,:,-2], Data[:,:,-1], -1)
# HOWTO assign a new "coordinate" var to an Xarray.DataArray
Data['WRVF'] = ('Sec',tmp)

n = Data.shape[-2]//2 - 60
tmp = hnanrvf( Data[:,:n,-2], Data[:,:n,-1],-1)
Data['WRVFa'] = ('Sec',tmp)
tmp = hnanrvf( Data[:,-n:,-2], Data[:,-n:,-1],-1)
Data['WRVFz'] = ('Sec',tmp)

Data.attrs['SortBy'] = ('WRVFz', True) # +++ ( coord, ascending=True)


In [ ]:
n = Data.shape[-2]//2 - 60
#< tmp = ( 100 - hpcorr( Data[:,-n:,-2:] ) * 100).round(0)
# tmp = hpcorr( Data[:,-n:,-2:] ).round(3)
tmp = hnancorr( Data[:,-n:,-1].values, Data[:,-n:,-2].values, -1 ).round(3)
Data['Pcorr'] = ('Sec',tmp)

#Data.attrs['SortBy'] = ('Pcorr', False) # +++


In [ ]:
n = Data.shape[-2]//2 - 60
#< tmp = ( 100 - hpcorr( Data[:,-n:,-2:] ) * 100).round(0)
tmp = hrcoef( Data[:,-n:,-1], Data[:,-n:,-2] )
Data['Rcoef'] = ('Sec',tmp.round(3))
#Data.attrs['SortBy'] = ('Rcoef', False) # +++


In [ ]:
# HOWTO sort Xarray by 
# Data = Data.sortby('WRVFz') # +++
Data.attrs['SortBy'] = ('WRVFz', True) # +++
#Data.attrs['SortBy'] = ('Pcorr', False) # +++
#Data.attrs['SortBy'] = ('Rcoef', False) # +++
#Data = Data.sortby( *Data.attrs['SortBy'] ) # +++


In [ ]:
#< Data['WRVFz'] = Data.WRVFz.round(3)
# Data['WRVFz'] = Data.WRVFz.round(3).values
#Data.WRVFz = Data.coords.WRVFz.round(3)
#Data.coords['WRVFz']
Data[Data.attrs['SortBy'][0]][:16].values

*******************************
# Boxplot Metric stats

In [ ]:
import seaborn as sns

In [ ]:
### [++-] Fig: Boxplots Su by Metrics
# HOWTO distribute Seaborn boxplots across mult axes:
tmp = pd.DataFrame.from_dict( { n:Data.coords[n] for n in ['Su','Se','WRVFz','Pcorr','Rcoef'] } )
tmp.WRVFz = -tmp.WRVFz

fig,ax = plt.subplots(1,3,figsize=(18,9))
#plt.subplots_adjust(wspace=0.4)
#< [ sns.boxplot( data=tmp.reset_index(), x='Su', y=['WRVFz','Pcorr','Rcoef'][n], ax=ax[n], whis=0) for n in range(ax.size)];
for n,m in enumerate(['WRVFz','Pcorr','Rcoef']):
    sns.boxplot( data=tmp.reset_index(), x='Su', y=m, ax=ax[n], whis=0 ).set(ylabel=None, title=m)

ax[0].set_ylim(-2,0); ax[1].set_ylim(-1,1);

ProcId = Pars[0]['ProcId'].split('_')[0]
#hsavefig(ProcId+'_BoxSuMetrics.png')


In [ ]:
### [++-] Fig: Boxplots Su by Metrics
# HOWTO distribute Seaborn boxplots across mult axes:
tmp = pd.DataFrame.from_dict( { n:Data.coords[n] for n in ['Su','Se','WRVFz','Pcorr','Rcoef'] } )
tmp.WRVFz = -tmp.WRVFz

# plt.style.use('default') # necessary reset?
fig,ax = plt.subplots(1,3,figsize=(4,6))
# HOWTO adjust axes spacing:
plt.subplots_adjust(wspace=0.6)
for n,m in enumerate(['WRVFz','Pcorr','Rcoef']):
    sns.boxplot( data=tmp.reset_index(), y=m, ax=ax[n] ).set(ylabel=None, xlabel=m)
    if n == 0: ax[0].axhline(-0.96,c='r')

ProcId = Pars[0]['ProcId'].split('_')[0]
#hsavefig(ProcId+'_BoxMetrics.png')


In [ ]:
tmp.quantile(0.95)


## Detailed boxplots of Su + Se + Ex

## Compare different ProcGroups if available

In [ ]:
### [++-] Fig: Boxplots by ProcGrp
# HOWTO make pd.DF from Xarray.DA.coords
tmp = pd.DataFrame.from_dict( { n:Data.coords[n] for n in ['Su','Se','Ex','Sc','ProcGrp','WRVFz','Pcorr','Rcoef'] } )
tmp.WRVFz = -tmp.WRVFz # for reverse sorting

x = tmp.ProcGrp.unique().tolist()
x = sorted(x)

m = ('WRVFz','Pcorr','Rcoef')[0] # not much difference!?!
fig,ax = plt.subplots(1,1,figsize=(4,6))
sns.boxplot( data=tmp.reset_index(), y=m, x='ProcGrp', order=x ).set(ylabel=None, title=m)
# sns.boxplot( data=tmp[tmp.Su==86].reset_index(), y='WRVFz', x='ProcGrp') #.set(ylabel=None, xlabel=m)
plt.setp(ax.get_xticklabels(), rotation=35);

# hsavefig('Su1Vr3+lin+Nt30+Tbat8_BoxWrvfz.png')


## Table of median per section


In [ ]:
### [+++] Save all metrics in a table as .csv
# FIXIT: somehow WRVF and WRVFa are identical!?!
df = xr.concat([ Data.coords[n] for n in ['Clipped','WRVF','WRVFa','WRVFz','Pcorr','Rcoef'] ], dim='Metric')
# df = xr.merge([ Data.coords[n] for n in ['Clipped','WRVF','WRVFa','WRVFz','Pcorr','Rcoef'] ])
df = df.to_pandas().T
df.columns = ['Clipped','WRVF','WRVFa','WRVFz','Pcorr','Rcoef']
# df[['Pcorr','Rcoef']] = -1 * df[['Pcorr','Rcoef']]
df = df.sort_index()
# df.to_csv('Km_D1Su1Vxn_SsesMetrics.csv') # SAVE to CSV
df

### Which Su data best predicts it's own left-out scans?


In [ ]:
### Create a table of metrics (coordinates)
tmp = pd.DataFrame.from_dict( { n:Data.coords[n] for n in ['Su','Se','Ex','WRVFz','Pcorr','Rcoef'] } )
tmp[['Pcorr','Rcoef']] = -1*tmp[['Pcorr','Rcoef']] # invert for sorting!
#tmp

In [ ]:
### Which Su data best predicts it's own left-out scans?
tmp.set_index(['Su','Se','Ex']).groupby(['Su']).median().idxmin()
# tmp.drop(['Se','Ex'],1).groupby(['Su']).median().idxmin() # same as above
tmp.drop(['Se','Ex'],1).groupby('Su').median().sort_values('WRVFz')


### Which experiments are best predicted by each subject's data?


In [ ]:
tmp.groupby(['Su','Se','Ex']).median().groupby('Su').idxmin()


# Plot best signals

### Top sections for 3 metrics

**************************************************

In [ ]:
# mpl.rcParams.update({'font.size': 10})
for n,m in enumerate([('WRVFz',True),('Pcorr',False),('Rcoef',False)]):
    # fig,ax = plt.subplots(2,3,figsize=(18,6))
    h = Data.sel(Sec=~Data.Clipped).sortby(*m)[:6,Data.shape[1]//2:,[0,1,-1,-2]].plot.line(
        x='t',col='Sec',col_wrap=3, figsize=(18,6), add_legend=False);
    [ plt.setp(n[-2:],'color','r') for n in h._mappables ];
    [ plt.setp(n[-1],'ls','','marker','.','ms',12,'color','r') for n in h._mappables ];
    tmp = (h.data.coords[m[0]].values*100).round(1)
    [ ax.set_ylabel('%s= %g%%'%(m[0],tmp[n])) for n,ax in enumerate(h.axes.flat)];
    plt.legend(Data.coords['Ch'].values[[0,1,-2,-1]]);
    #hsavefig(ProcId+'_Top6'+m[0])
    print(ProcId+'_Top6'+m[0]+'.png')


### Top 12 sections for XXX metric

In [ ]:
# m = [('WRVFz',True),('Pcorr',False),('Rcoef',False)][0] # *** choose Pcorr
# h = Data.sortby(*m)[:12,Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,12), add_legend=False);
# h = Data.sel(Sec=~Data.Clipped).sortby(*m)[:np.prod(RC),Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=RC[1], figsize=(18,14), add_legend=False);
def hPlotXYYh1( Data ):
    h = Data.plot.line(x='t',col='Sec', col_wrap=3, figsize=(18,14), add_legend=False);
    # h = Data.sel(Sec=~Data.Clipped).sortby(*m).sel(Su=63)[:12,Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,12), add_legend=False);
    [ plt.setp(n[-2:],'color','r') for n in h._mappables ];
    [ plt.setp(n[-2],'ls','','marker','.','ms',12,'color','r') for n in h._mappables ];
    tmp = (h.data.coords[m[0]].values*100).round(1)
    [ ax.set_ylabel('%s= %g%%'%(m[0],tmp[n])) for n,ax in enumerate(h.axes.flat)];
    plt.legend(Data.coords['Ch'].values[[0,1,-2,-1]]); # 

    # hsavefig( ProcId+'_Top12WRVFz.png')


In [ ]:
# m = [('WRVFz',True),('Pcorr',False),('Rcoef',False)][0] # *** choose Pcorr
# h = Data.sortby(*m)[:12,Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,12), add_legend=False);
def hPlotXYYh( Data, Sort=('WRVFz',True), RC=(5,3)):
    m = Sort
    h = Data.sel(Sec=~Data.Clipped).sortby(*m)[:np.prod(RC),Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=RC[1], figsize=(18,14), add_legend=False);
    # h = Data.sel(Sec=~Data.Clipped).sortby(*m).sel(Su=63)[:12,Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,12), add_legend=False);
    [ plt.setp(n[-2:],'color','r') for n in h._mappables ];
    [ plt.setp(n[-2],'ls','','marker','.','ms',12,'color','r') for n in h._mappables ];
    tmp = (h.data.coords[m[0]].values*100).round(1)
    [ ax.set_ylabel('%s= %g%%'%(m[0],tmp[n])) for n,ax in enumerate(h.axes.flat)];
    [ ax.spines[n].set_visible(False) for ax in h.axes.flat for n in ('top','right')];
    plt.legend(Data.coords['Ch'].values[[0,1,-2,-1]]); # 

    # hsavefig( ProcId+'_Top12WRVFz.png')
    
    return h


In [ ]:
# HOWTO remove box i.e. right and top axes ("spines") from MPL axes.
hboxoff = lambda ax,loc=('top','right'): [ n.spines[m].set_visible(False) for n in ax for m in loc ]


In [ ]:
if np.unique(Data.ProcGrp).size > 1: 
    warn('Multiple ProcGrp in Data.')
    ProcId = np.unique(Data.ProcGrp)[0]
ProcId

In [ ]:
m = [('WRVFz',True),('Pcorr',False),('Rcoef',False)][0] # *** choose Pcorr
# hPlotXYYh( Data, Sort=m )
hPlotXYYh( Data.sel(ProcGrp=ProcId), Sort=m )

#hsavefig(ProcId+'_Top15'+m[0])
print(ProcId+'_Top15'+m[0]+'.png')



### Bottom 12 sections for XXX metric

In [ ]:
m = [('WRVFz',True),('Pcorr',False),('Rcoef',False)][1] # *** choose Pcorr
if False: # Plot sections closest to median (abs)
    h = Data.sel(Sec=~Data.Clipped).sortby(np.abs(Data[m[0]]-Data[m[0]].median()))[:12,Data.shape[1]//2:,[0,1,-2,-1]]\
        .plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,12), add_legend=False);
else: # Plot sections closest to zero (abs) <-- Pcorr
    assert 'WRVF' not in m[0], 'Oops! These are not uncorrelated.'
    h = Data.sel(Sec=~Data.Clipped).sortby(np.abs(Data[m[0]]))[:12,Data.shape[1]//2:,[0,1,-2,-1]]\
        .plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,12), add_legend=False);
[ plt.setp(n[-2:],'color','r') for n in h._mappables ];
[ plt.setp(n[-2],'ls','','marker','.','ms',12,'color','r') for n in h._mappables ];
tmp = (h.data.coords[m[0]].values*100).round(1)
[ ax.set_ylabel('%s= %g%%'%(m[0],tmp[n])) for n,ax in enumerate(h.axes.flat)];
plt.legend(Data.coords['Ch'].values[[0,1,-2,-1]]); # 

F = '%s_Bot12%s.png'%(ProcId,m[0])
print(F)
# hsavefig(F,False)


### Top 3 by *metric* for each Subject

In [ ]:
idx = Data.sel(ProcGrp='Su1Vx7').Sec
plt.hist( [ Data.WRVFz.sel(ProcGrp='Su0Vs1'), Data.WRVFz.sel(ProcGrp='Su1Vx7')], bins=100)
plt.legend(np.unique(Data.ProcGrp));

In [ ]:
### Fig: Top 3 Pcorr for each Subject
m = [('WRVFz', True),('Pcorr',False),('Rcoef',False)][0] # *** choose Pcorr
ProcId = ProcId.split('_')[0]
for n in np.unique(Data.Su):
    h = Data.sel(Sec=~Data.Clipped).sel(Su=n).sortby(*m)[:3,Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,3), add_legend=False);
    # Pcorr closest to zero:
    # h = Data.sel(Sec=~Data.Clipped).sortby(np.abs(Data[m[0]])).sel(Su=n)[:3,Data.shape[1]//2:,[0,1,-2,-1]].plot.line(x='t',col='Sec',col_wrap=3, figsize=(18,3), add_legend=False);
    [ plt.setp(n[-2:],'color','r') for n in h._mappables ];
    [ plt.setp(n[-2],'ls','','marker','.','ms',12,'color','r') for n in h._mappables ];
    tmp = (h.data.coords[m[0]].values*100).round(1)
    [ ax.set_ylabel('%s= %g%%'%(m[0],tmp[n])) for n,ax in enumerate(h.axes.flat)];
    hboxoff( h.axes.flat )
    plt.legend(Data.coords['Ch'].values[[0,1,-2,-1]]);
    
    F = '%s_Top3%sSu%03u.png'%(ProcId,m[0],n)
    print(F)
    hsavefig(F,False)

    # break

# HOWTO joing images vertically with ImageMagick
# !convert -append *.png tmp.png # stitch all together


----------------------------------------------------

----------------------------------------------------

******************************************
# Carpet plots?

In [ ]:
tmp = Data[:,:,-2:].diff('Ch').squeeze(drop=True)
# tmp = Data[:,:,-1].squeeze(drop=True)
tmp = tmp.where(~np.isnan(tmp),drop=True)
# tmp = tmp.where(~np.any( np.isnan(tmp),0),drop=True)
# tmp = tmp[:,~np.any( np.isnan(tmp),0)]
tmp.shape

In [ ]:
tmp = Data.WRVFz
tmp = Data.Rcoef
tmp = Data.Pcorr
tmp = tmp.sortby('Sec')

In [ ]:
# tmp.sortby('Sec').reset_index('Sec').plot.line()
tmp.reset_index('Sec').plot.line()
#np.nonzero( tmp['Su'].diff(0))
#hvline( np.flatnonzero( np.diff(tmp.Su.values))+1)
plt.vlines(np.flatnonzero( np.diff(tmp.Su.values))+1,*plt.ylim(),ls=':');
plt.hlines(1,*plt.xlim(),'r');
np.unique(tmp.Su)

******************************************

**********************************************************
# Scratch

Looks like Su 65 and 6 (or 78?) show the best results on these metrics.

### HOWTO make a MultiIndex for Su*Sec - mult. alternatives: